## The task consist to classify the semantic of tweets. To deal with a deep understanding of natural language I'll try directly to handle the task with the transformer achitecture.


### I'll use the HF library

In [ ]:
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    datasets \
    evaluate --quiet

%pip install accelerate -U --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset
from transformers import  AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
def from_txt_to_pd(files_dir):
    categories = ['neg', 'pos']

    df = pd.DataFrame(columns=['text','sentiment'])

    for category in categories:
        path = os.path.join(files_dir, category)
        for txt in os.listdir(path):
            txt_path = os.path.join(path, txt)
            if os.path.getsize(txt_path) > 0:
                label = categories.index(category)
                file = open(txt_path, "r")
                text = file.read()
                file.close()
                new = [text, label]
                # New data as pandas.DataFrame
                new = pd.DataFrame(columns=df.columns, data=[new])
                df = pd.concat([df, new], ignore_index=True)
    return df

In [ ]:
file_dir = '/kaggle/input/movie-reviews/aclImdb/train/'
train_df = from_txt_to_pd(file_dir)

In [ ]:
train_df.head()

,text,sentiment
0,"This film is the worst film, but it ranks very...",0
1,"I should never have started this film, and sto...",0
2,I'm here again in your local shopping mall (of...,0
3,Black and White film. Good photography. Believ...,0
4,from the start of this movie you soon become a...,0


In [ ]:
train_df.groupby(['sentiment']).count()

,text
sentiment,
0,12500
1,12500


In [ ]:
file_dir = '/kaggle/input/movie-reviews/aclImdb/test/'
test_df = from_txt_to_pd(file_dir)

### Check if it's running on the GPU

In [ ]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


### After I made some tests, I wrote a two functions to handle the data cleaning, by removing part of the text that cannot be handle well by the tokenizer

In [ ]:
import re

def text_clean(text):

    ### Light
    #text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    #text = re.sub(r'%20', '', text)
    text = text.replace("%20", " " )
    text = re.sub(r'\'\w+', '', text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'<br />', ' ', text)


    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()


    return text

In [ ]:
def prepare_promt(df_to_clean):
    df_to_clean['text'] = df_to_clean.text.apply(text_clean)
    return df_to_clean

### Now call the function to clean the text

In [ ]:
train_df_cleaned = prepare_promt(train_df)
test_df_cleaned = prepare_promt(test_df)

In [ ]:
print(train_df.text[554])
print(train_df_cleaned.text[554])
print(train_df.text[67])
print(train_df_cleaned.text[67])

i know you read that before, on countless other films no doubt if you reading the comments here, but voodoo academy still stands as the absolute worst film i been able to track down. no doubt the really bad ones aren even available to buy or watch on tv, but even so i feel it fair to qualify that i not just some dumb renter who picked badly. i seen two thirds of the bottom ranked films here on the imdb, and i ticking of the remaining ones with every chance. most of those stand head and shoulders above this... excersize in absolute monotony. i like to rate truly bad films (as in ones with no humour even in how bad they are) by how many people you need to watch it with to make it all the way through. if you can watch it by yourself, it isn that bad. if you can watch it with one single friend... it bad but could be a lot worse. it took of us to make it all the way through voodoo academy. and not even the usually fun moments of watching bad films (spotting the boom mic for example, times i

{'text': 'I have to start by telling you how I came across this movie.It was winter time in Alaska around the year .A friend of mine from Australia was staying with me and my girl friend in a shoe box of an apartment.Winters in Alaska can be a bit brutal and most people stay indoors,drink heavily and watch anything that comes on the television.I had found this movie outside of a thrift store laying in a snowbank and right away new it was a treasure.It is quite possibly the best worst movie ever.We spent the next two weeks watching this movie and drinking like fish.We watched it so many times in fact that we would sometimes turn the television on its side or upside down for a more full filling effect.It is a true gem.The laughs will come nonstop and the memories last forever.If you see this movie for rent in a video store,steal it.You won regret it!', 'label': 1}


### It seems fine. Now it's time to create a HF dataset since to utilize LLM models with HF it's required

In [ ]:
from datasets import Dataset

train_df_cleaned.rename(columns={'sentiment': 'label'}, inplace=True) # the transformer model require that the Y is called label


dataset = Dataset.from_pandas(train_df_cleaned) # load Dataset from Pandas DataFrame

dataset = dataset.train_test_split(test_size=0.2) # I split the dataset in two so that I can use an unseen part as evaluation data

print(dataset)

print(dataset["train"][100])
print(dataset["test"][10])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})
{'text': 'In this movie, Chvez supporters (either venezuelan and not-venezuelan) just lie about a dramatic situation in our country. They did not say that the conflict started because of Chvez announcement firing a lot of PDVSA best workers just for political issues. They did not say anything about more than TV interruptions transmitted by Chvez during only days in "CADENA NACIONAL" (a kind of confiscation o private TV signals). Each one with about minutes of duration. They did not tell us anything about The quiting announcement made by General en Jefe Lucas Rincon Romero, Inspector General of the army forces, who is a traditional supporter of Chvez. Even now, in despite of his announcement, he is the Ministro de Interior y Justicia. After Chvez return he occuped the Charge of Ministro del Defens

### To finetune the llm I need to use a tokenized version of the prompt, and it needs to be in the same encoding way as it the model was pretrained

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### First to check how well the model is working I train it on a small part of the dataset. This way I have a faster way to try it with differt parameters and prompt format. And also important to not waste computational resurces

In [ ]:
light_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
light_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
bs = 18
epochs = 3
lr = 2e-5

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
from transformers import logging
logging.set_verbosity_error() # to avoid warning messages

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
trainer = Trainer(model, args, train_dataset=light_train_dataset, eval_dataset=light_eval_dataset,
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.we

In [ ]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.433561,0.870000
2,No log,0.299047,0.882000
3,No log,0.316340,0.876000


### the evaluation result seems pretty good, I can try it on the test set and see if it maintains similar results

### And as explained previously it needs to be encoded also in the same way

In [ ]:
test_ds = Dataset.from_pandas(test_df_cleaned).map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

### Perform the prediction

In [ ]:
preds_light = trainer.predict(test_ds).predictions
preds_light

array([[ 1.2911031 , -1.564731  ],
       [-0.666106  ,  0.55655736],
       [ 1.3999647 , -1.8256439 ],
       ...,
       [ 1.4486115 , -1.8008753 ],
       [-1.2210609 ,  1.176431  ],
       [-1.0448589 ,  0.9568001 ]], dtype=float32)

### And I need to extract the predictions from the 2D array

In [ ]:
preds_light = np.argmax(preds_light, axis=-1)
preds_light

array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
y_real = test_df_cleaned['sentiment']
y_real = y_real.to_list()
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_real, preds_light)
print('Accuracy: %f' % accuracy)

Accuracy: 0.874560


### I have an accuracy of circa 87% that is pretty good, now I can perform the finetuning on the full dataset

In [ ]:
bs_full = 18
epochs_full = 3
lr_full = 2e-5

In [ ]:
args_full = TrainingArguments('outputs', learning_rate=lr_full, warmup_ratio=0.1,
    evaluation_strategy="epoch", per_device_train_batch_size=bs_full, per_device_eval_batch_size=bs_full*2,
    num_train_epochs=epochs_full, weight_decay=0.01, report_to='none')

In [ ]:
model_full = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
trainer_full = Trainer(model_full, args_full, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["test"],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.we

In [ ]:
trainer_full.train();

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.241900,0.227941,0.911600
2,0.165400,0.233922,0.921000
3,0.096900,0.310880,0.922000


### I can see that now I have a better accuracy, so I can perform again the prediction, this time with the new model and submit it

In [ ]:
preds_full = trainer_full.predict(test_ds).predictions
preds_full = np.argmax(preds_full, axis=-1)

In [ ]:
y_real = test_df_cleaned['sentiment']
y_real = y_real.to_list()
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_real, preds_full)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_real, preds_full)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_real, preds_full)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_real, preds_full)
print('F1 score: %f' % f1)


Accuracy: 0.929520
Precision: 0.930484
Recall: 0.928400
F1 score: 0.929441


### I have an accuracy of 93% on the test set